# Reparameterizing the law

The empirical law as it is demonstrates that iron exerts a large degree of control over the conductivity, up to 2 log units below $\approx$ 900C. Reparameterizing the law to take into account specific conduction mechanisms allows closer comparison and integration with other data. It will also allow folding of the data into a single law rather than two different ones. 

Conductivity is assumed in this model to vary similar to the SEO3 model by Constable (2006), in that Fe and Mg vacancies are taken into account. 

$$\sigma = [Fe^\cdot_{Mg}]\mu_{Fe} + [V^\cdot_{Mg}]\mu_{Mg} + [?]\mu_0$$

One difference is that here, we include Fe# and Mg# as part of the concentration model to apply mineralogic information into the crystal terms.

$$= \alpha(Fe\#)\mu_{Fe} + \beta(Mg\#)\mu_{Mg}$$

For each conduction mechanism, we assume that there exists a term dependent on the 1/6 power of oxygen fugacity, and one not dependent on it. The 1/6 exponent law comes from an olivine-specific defect analysis and therefore may be innappropriate. Only weak dependencies on fO2 are expected in any case. Both are multiplied by the iron and magnesium number.

$$= (Fe\#)\cdot[a_{Fe}(T) + b_{Fe}(T) fO_2^{1/6}]\mu_{Fe} + (Mg\#)\cdot[a_{Mg}(T) + b_{Mg}(T) fO_2^{1/6}]\mu_{Mg}$$

Assume now that all constants follow an arrhenious law

$$= (Fe\#)\cdot[a_{Fe}e^{\alpha_{Fe}/kT} + b_{Fe}e^{\beta_{Fe}/kT} fO_2^{1/6}]\mu_{Fe} + (Mg\#)\cdot[a_{Mg}e^{\alpha_{Mg}/kT} + b_{Mg}e^{\beta_{Mg}/kT} fO_2^{1/6}]\mu_{Mg}$$

If the mobilities also follow an arrhenious law, we can assume any fit will be folded into their exponents.

$$= (Fe\#)\cdot[a_{Fe}e^{\alpha_{Fe}/kT} + b_{Fe}e^{\beta_{Fe}/kT} fO_2^{1/6}] + (Mg\#)\cdot[a_{Mg}e^{\alpha_{Mg}/kT} + b_{Mg}e^{\beta_{Mg}/kT} fO_2^{1/6}]$$

We now have 8 unknowns, $a_{Fe,Mg}, b_{Fe,Mg}, \alpha_{Fe,Mg}, \beta_{Fe,Mg}$

The fit I will try will come from exploring a range of temperatures and corresponding oxygen fugacities from the DK7 and MAL samples from Hueber '88. 

In [ ]:
#first 1000 temperatures 
import pandas as pd
from scipy.optimize import curve_fit

temps = np.linspace(800,1600,num=100)
logO2 = calc_QFM(temps,0)
xIron1 = np.ones(len(temps))*pyroxene_law1.xfe
xIron2 = np.ones(len(temps))*pyroxene_law2.xfe
conductivity1 = np.log10(pyroxene_law1.isotropic_conductivity(temperature=temps+273,pressure=0,extrapolate=True))
conductivity2 = np.log10(pyroxene_law2.isotropic_conductivity(temperature=temps+273,pressure=0,extrapolate=True))
data_1 = {'temperature':temps,'logO2':logO2,'xFe':xIron1,'sigma':conductivity1}
data_2 = {'temperature':temps,'logO2':logO2,'xFe':xIron2,'sigma':conductivity2}
df = pd.concat([pd.DataFrame(data_1),pd.DataFrame(data_2)],ignore_index=True)
y_data = df['sigma']

def fit(x, afe, bfe, alphafe, betafe, amg, bmg, alphamg, betamg,c0,omega):
    k      = 8.617e-5 
    temp   = x[0]
    fO2    = 10**x[1]
    xfe    = x[2]
    
    iron_term  = xfe*((10**afe)*np.exp(-alphafe/(k*temp))     +\
                      (10**bfe)*np.exp(-betafe/(k*temp))*fO2**(1/6))
    mg_term    = (1-xfe)*((10**amg)*np.exp(-alphamg/(k*temp)) + \
                          (10**bmg)*np.exp(-betamg/(k*temp))*fO2**(1/6))
    extra_term = (10**c0)*np.exp(-omega/(k*temp))
    
    return np.log10(iron_term + mg_term + extra_term)

def fit_qfm(temp, xfe, afe, bfe, alphafe, betafe, amg, bmg, alphamg, betamg, c0, omega):
    k   = 8.617e-5 
    fO2 = 10**calc_QFM(temps,0)
    iron_term  = xfe*((10**afe)*np.exp(-alphafe/(k*temp))     +\
                      (10**bfe)*np.exp(-betafe/(k*temp))*fO2**(1/6))
    mg_term    = (1-xfe)*((10**amg)*np.exp(-alphamg/(k*temp)) + \
                          (10**bmg)*np.exp(-betamg/(k*temp))*fO2**(1/6))
    extra_term = (10**c0)*np.exp(-omega/(k*temp))
    return np.log10(iron_term + mg_term + extra_term)
    
xdata = df.loc[:,['temperature','logO2','xFe']].values.T
popt, pcov,infodict,mesg,ier = curve_fit(fit, xdata, y_data, maxfev=20000,full_output=True,xtol=1e-32)         

plt.plot(temps,conductivity2)
plt.plot(temps,fit_qfm(temps, pyroxene_law1.xfe, *popt))
mesg